### get_GST_from_QKG_with_Frozenset.py
### def call_main_GST(f1, f2, f4, f5, no_GST, gdict, verbose, gt1, config, context_file):
### graph_node,graph_edge,GST_match_flag, candidate_match_flag1, top_match_flag1, candidate_match_flag2, top_match_flag2, candidate_match_flag3, top_match_flag3, candidate_match_flag4, top_match_flag4, candidate_match_flag5, top_match_flag5 = call_main_GST(QKG_file, cornerstone_file, qtype, answer_list_file, no_GST, gdict, verbose,gt,config,con)
### call_main_GST(QKG_file, cornerstone_file, qtype, answer_list_file, no_GST, gdict, verbose,gt,config,con)
    f1, f2, f4, f5, no_GST, gdict, verbose, gt1, config, context_file
        f1=argv[1] #input QKG
        f2=argv[2] #input Cornerstones
        f4=argv[3] #answer match / qtype
        f5=argv[4] #answer type /answer_list_file
        no_GST=int(argv[5]) #number of GSTs
        
        verbose=int(argv[6])
    QKG_file, cornerstone_file, qtype, answer_list_file, no_GST, gdict, verbose,gt,config,con

In [1]:
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
# from stanfordcorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP(r'stanford-corenlp-full-2018-10-05')

In [2]:
import networkx as nx
from heapq import heappush, heappop, heapify
import Queue
import pickle
import sys
import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
Fpath='/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/'

In [4]:
f1 = Fpath+"files/QKG_ques-q1"
f2 = Fpath+"files/QKG_cornerstones_ques-q1"
f4 = Fpath+"question_type.txt"
f5 = Fpath+"files/Answer_list_ques-q1"
no_GST = 50 #Number of GSTs to be considered 
# if embedding=='WORD2VEC':
# gdict={}
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  
# word_vectors = model.wv
# gdict=word_vectors

verbose = 0
gt = set(['unknown'])

stream=open("config.yml", 'r')

# for line in stream:
#     if not line.startswith('#') and len(line)>1:
#         #print 'line ',line
#         line=line.split(':')
#         config[line[0]]=line[1].strip()

con=Fpath+"files/context_ques-q1"

In [5]:
config={}
for line in stream:
    if not line.startswith('#') and len(line)>1:
        #print 'line ',line
        line=line.split(':')
        config[line[0]]=line[1].strip()

In [6]:
gdict={}
model = gensim.models.KeyedVectors.load_word2vec_format(Fpath+'GoogleNews-vectors-negative300.bin.gz', binary=True)  
word_vectors = model.wv
gdict=word_vectors

/usr/local/anaconda3/envs/QUEST/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
gdict

In [8]:
threshold = float(config['Cosine_threshold'])
MAX_MATCH = int(config['Embedding_Similarity'])
# Type filtering can be none (0), Relaxed i.e. only apply if candidate list greater than 10 (1), or Strict i.e. apply always (2)
Type_Filtering_flag = int(config['Type_Filtering'])
Distribute_Node_wt_flag = int(config['Distribute_Node_wt'])
chain_join_flag = int(config['chain_join_flag'])

In [9]:
threshold

0.75

In [10]:
# try:
G1 = nx.read_gpickle(f1)
if verbose:
    print "\n\nSize of the read graph ", len(G1.nodes()), len(G1.edges())
corner1 = pickle.load(open(f2, 'r'))

# except:
#     if verbose:
#         print "No graph or Cornerstones\n"
#     answer_list = []
#     pickle.dump(answer_list, open(f5, 'w'))
#     return

In [11]:
corner1

{u'11 steve jobs:Entity': 'steve jobs',
 u'1985 jobs:Entity': 'steve jobs',
 u'1986 jobs:Entity': 'steve jobs',
 u'2003 jobs:Entity': 'steve jobs',
 u'2009 jobs:Entity': 'steve jobs',
 u'actor steve carell:Entity': 'steve jobs',
 u'adoption jobs:Entity': 'steve jobs',
 u'after jobs:Entity': 'steve jobs',
 u'apple co-founder 06 oct 2011 steve jobs:Entity': 'steve jobs',
 u'apple co-founder steve jobs:Entity': 'steve jobs',
 u'apple jobs:Entity': 'steve jobs',
 u'biological father steve jobs:Entity': 'steve jobs',
 u'birth after:Predicate:1': 'born',
 u'birth family during:Predicate:1': 'born',
 u'birth father:Predicate:1': 'born',
 u'birth mother out:Predicate:1': 'born',
 u'book steve jobs 2011:Entity': 'steve jobs',
 u'book steve jobs:Predicate:1': 'steve jobs',
 u'born in:Predicate:1': 'born',
 u'born in:Predicate:2': 'born',
 u'born in:Predicate:3': 'born',
 u'born in:Predicate:4': 'born',
 u'born in:Predicate:5': 'born',
 u'born in:Predicate:6': 'born',
 u'born in:Predicate:7': 'bo

In [12]:
corner = corner1
if verbose:
    print "\n\nSize of the unconverted directed graph, Number of cornerstones ", len(G1.nodes()), len(
        G1.edges()), len(corner)
# G=G1.to_undirected() #make QKG Undirected
# G=directed_to_undirected(G1)
G = G1

In [13]:
def initialize_queue(G, corner):
    Q = []
    T = {}
    for v in corner: # corner -> v:p (v is index, p is Corresponding value(problem entity)) corner is a set
        T[v] = {}
        g = nx.Graph()
        g.add_node(v)  # corner中的点，access weight from G if needed, graph with 1 node, entity node
        p = frozenset([corner[v]])  # Query term       corner[v] is a problem node
        T[v][p] = g # T -> v:p:graph(one node->entity node) -> u'birth after:Predicate:1': {frozenset({'born'}): <networkx.classes.graph.Graph at 0x258cfc9d0>},
        c = get_cost(g, G)# g: only one node in graph g #计算t和G，边重叠部分的weight（1-wlist）, wlist：edges' weight of two nodes
        heappush(Q, (c, v, p))

    for v in T:
        if verbose:
            print "Vertex ", v
        for p in T[v]:
            if verbose:
                print "Query and tree --->", p, T[v][p].nodes(), T[v][p].edges()
    if verbose:
        print "Queue --->", Q
    return T, Q

In [14]:
def get_cost(t, G): #1-weight的加和
    c = 0.0
    for (n1, n2) in t.edges():###计算t和G，边重叠部分的weight（1-wlist）, wlist：edges' weight of two nodes
        if (n1, n2) in G.edges():
            data = G.get_edge_data(n1, n2)
        else:
            data = G.get_edge_data(n2, n1)
        #n1 & n2 existe in t&G, get cost=1-(weight1 +weight2) weight1 & weight2 in data['wlist']
        for d in data['wlist']:  # Use if sum of cost is needed
            # d=data['weight']
            c += 1 - d  # cost=1-(weight1 +weight2) for 1 pair, sum all c in here.
    # if d==0:
    #	print "Edge d 0",data['weight'],data['wlist'],data['etype'],n1,n2
    # c+=1-d
    if c < 0:
        if verbose:
            print "\n\n ==== =========== ERROR NEG COST ===== \n\n", c
#Distribute node weights to edge weights; 
#0 means no distribution, 不进行分发节点weight工作
#1 means using joint optimization, #联合优化分发weight
#2 means adding half of endnode weights to the edge-weight. 1/2weight

    if Distribute_Node_wt_flag == 1: ##only use node's weight
        for n in t.nodes(): #只计算t的weight
            node_weight = G.node[n]['weight']
            # if node_weight>1:
            #	node_weight=1
            c += 1 - node_weight #c = 1 - node_weight for 1 node. sum all c for all node weight.
        if c < 0:
            if verbose:
                print "\n\n ==== =========== ERROR NEG COST Node===== \n\n", c

    return c

In [15]:
if verbose:
    print "\n\nSize of the converted undirected graph largest connected component, Number of cornerstones ", len(
        G.nodes()), len(G.edges()), len(corner), corner

T, Q = initialize_queue(G, corner)
P = set()  # Entire query
for v in corner:
    P.add(corner[v])

if verbose:
    print P

count = {}
for v in corner:
    if corner[v] not in count:
        count[corner[v]] = set()
    count[corner[v]].add(v)

if verbose:
    print "\n\nCorner stone count per query term -->", count

In [16]:
T#node in corner 
# corner -> v:p (v is index, p is Corresponding value(problem node)) corner is a set
# T -> v:p:graph(one node) -> u'birth after:Predicate:1': {frozenset({'born'}): <networkx.classes.graph.Graph at 0x258cfc9d0>},

{u'11 steve jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d1380d0>},
 u'1985 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b090>},
 u'1986 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138b10>},
 u'2003 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b250>},
 u'2009 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b5d0>},
 u'actor steve carell:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138c10>},
 u'adoption jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138e50>},
 u'after jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138190>},
 u'apple co-founder 06 oct 2011 steve jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d15e090>},
 u'apple co-founder steve jobs:Entity': {frozenset({'steve jobs'}): <n

In [17]:
v

u'born in:Predicate:5'

In [18]:
corner[v]

'born'

In [19]:
frozenset([corner[v]])

frozenset({'born'})

In [20]:
T[v]

{frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d15ed90>}

In [21]:
T[v][frozenset([corner[v]])]

In [22]:
T[v][frozenset([corner[v]])].edges()

[]

In [23]:
T[v][frozenset([corner[v]])].nodes()

[u'born in:Predicate:5']

In [25]:
v_test0='1958:Entity'

In [26]:
v_test1=unicode(v_test0, 'utf-8') 

In [27]:
v_test1

u'1958:Entity'

In [28]:
T[v_test1]

KeyError: u'1958:Entity'

In [29]:
corner[v_test1]

KeyError: u'1958:Entity'

In [30]:
Q

[(0.0, u'11 steve jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'2009 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'1985 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'after jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'apple jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'1986 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'2003 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'birth mother out:Predicate:1', frozenset({'born'})),
 (0.0, u'birth after:Predicate:1', frozenset({'born'})),
 (0.0, u'born in:Predicate:3', frozenset({'born'})),
 (0.0, u'born in:Predicate:5', frozenset({'born'})),
 (0.0, u'born on:Predicate:3', frozenset({'born'})),
 (0.0, u'actor steve carell:Entity', frozenset({'steve jobs'})),
 (0.0, u'adoption jobs:Entity', frozenset({'steve jobs'})),
 (0.0,
  u'apple co-founder 06 oct 2011 steve jobs:Entity',
  frozenset({'steve jobs'})),
 (0.0, u'born in:Predicate:4', frozenset({'born'})),
 (0.0, u'book steve jobs 2011:Entity', frozenset({'steve

In [31]:
count

{'born': {u'birth after:Predicate:1',
  u'birth family during:Predicate:1',
  u'birth father:Predicate:1',
  u'birth mother out:Predicate:1',
  u'born in:Predicate:1',
  u'born in:Predicate:2',
  u'born in:Predicate:3',
  u'born in:Predicate:4',
  u'born in:Predicate:5',
  u'born in:Predicate:6',
  u'born in:Predicate:7',
  u'born in:Predicate:8',
  u'born into:Predicate:1',
  u'born on:Predicate:1',
  u'born on:Predicate:10',
  u'born on:Predicate:2',
  u'born on:Predicate:3',
  u'born on:Predicate:4',
  u'born on:Predicate:5',
  u'born on:Predicate:6',
  u'born on:Predicate:7',
  u'born on:Predicate:8',
  u'born on:Predicate:9',
  u'born out:Predicate:1',
  u'born:Predicate:1',
  u'born:Predicate:2',
  u'born:Predicate:3',
  u'born:Predicate:4',
  u'born:Predicate:5',
  u'born:Predicate:6',
  u'born:Predicate:7',
  u'born:Predicate:8',
  u'born:Predicate:9',
  u'british-born industrial designer:Entity',
  u'emigrated from:Predicate:1',
  u'married because:Predicate:1',
  u'married in

In [32]:
P

{'born', 'steve jobs'}

In [33]:
corner

{u'11 steve jobs:Entity': 'steve jobs',
 u'1985 jobs:Entity': 'steve jobs',
 u'1986 jobs:Entity': 'steve jobs',
 u'2003 jobs:Entity': 'steve jobs',
 u'2009 jobs:Entity': 'steve jobs',
 u'actor steve carell:Entity': 'steve jobs',
 u'adoption jobs:Entity': 'steve jobs',
 u'after jobs:Entity': 'steve jobs',
 u'apple co-founder 06 oct 2011 steve jobs:Entity': 'steve jobs',
 u'apple co-founder steve jobs:Entity': 'steve jobs',
 u'apple jobs:Entity': 'steve jobs',
 u'biological father steve jobs:Entity': 'steve jobs',
 u'birth after:Predicate:1': 'born',
 u'birth family during:Predicate:1': 'born',
 u'birth father:Predicate:1': 'born',
 u'birth mother out:Predicate:1': 'born',
 u'book steve jobs 2011:Entity': 'steve jobs',
 u'book steve jobs:Predicate:1': 'steve jobs',
 u'born in:Predicate:1': 'born',
 u'born in:Predicate:2': 'born',
 u'born in:Predicate:3': 'born',
 u'born in:Predicate:4': 'born',
 u'born in:Predicate:5': 'born',
 u'born in:Predicate:6': 'born',
 u'born in:Predicate:7': 'bo

In [34]:
verbose

0

In [35]:
def get_GST(Q, T, P, G, no_GST, corner, verbose):
    pop_hist = {}
    ite = 0
    pop_cov = -1
    merge_cov = -1
    min_cost = 999999
    GST_count = 0
    GST_set = []
    final_GST_cost = -99999
    final_GST_flag = 0
    leave_loop = 0

    while len(Q) > 0:
        x = heappop(Q)#pop the minimize in Q 

        cc = x[0] #cost
        v = x[1] #node in corner
        p = x[2] #problem node

        if verbose:
            print "======================================================================================>>>>>>>>>"
        if verbose:
            print "iteration ->", ite
        ite += 1
        if verbose:
            print "\n\nPopped --->", x[0], x[1], x[2], len(p.intersection(P)), len(P), len(Q), pop_cov, merge_cov, min_cost
        if len(p) > pop_cov:
            pop_cov = len(p) #the length of problem
            if verbose:
                print "pop length increased to ", len(p)

        check = check_history(pop_hist, v, p)#if v(entity) in pop_hist and p(problem) in pop_hist[v], return 1
        if check == 0:
            continue  # To avoid executing same pop
        if v not in pop_hist:
            pop_hist[v] = set()
        pop_hist[v].add(p)

        if p == P:
            current_cost = get_cost(T[v][p], G) #T[v][p]只有一个点的图？？？
            # print '\nCurrent Cost ',current_cost,len(GST_set),GST_count,v,p
            # for (n1,n2) in T[v][p].edges():
            # data=G.get_edge_data(n1,n2)
            # print "Tree edge ",n1,n2,data['weight'],data['wlist'],data['etype']
            if final_GST_cost > -99999 and current_cost > final_GST_cost:  # No more GST with same cost as final GST
                leave_loop = 1
                break

            GST_set, New_GST_flag, pot_ans_flag = save_GST(GST_set, v, T[v][p], corner, G) #v:entity, T:grpah, corner, G:total graph
            if New_GST_flag == 1 and pot_ans_flag == 1:
                GST_count += 1
            if GST_count == no_GST and final_GST_flag == 0: #expanding 5 times
                final_GST_cost = get_cost(T[v][p], G)
                final_GST_flag = 1

        if leave_loop == 1:
            break
        # GROW #add neighbors' edges and nodes + itself
        for u in G.neighbors(v):#
            if verbose:
                print "\nneighbor ", u, T[v][p].nodes(), T[v][p].edges(), len(Q)
            # gflag=0
            g = grow_graph(T[v][p], v, u)  # Crete temporary merge tree/graph | v:node in corner. u:v's neighbor
            if verbose: #g是v本身和v-u边
                print "\nGrown ", v, p, g.nodes(), g.edges(), len(Q)
            cg = get_cost(g, G)#g‘s cost
            flag = 0
            if u in T and p in T[u]:#neighbor的点在corner中
                cu = get_cost(T[u][p], G) #neibor本身和G
                if cg < cu:  # If T[u][p] already exists, check if cost(merged_graph)<T[u][p]; in that case update T[u][p]
                    if verbose:#比较自身v和neibor，u的影响力，如果自身影响力大的话把邻居graph扩大
                        print "\nBefore Growth updated ", u, p, cg, cu, T[u][p].nodes(), T[u][p].edges(), get_cost(
                            T[u][p], G), len(Q)
                    T[u][p] = g
                    flag = 1
                    if verbose:
                        print "\nAfter Growth updated ", u, p, cg, cu, T[u][p].nodes(), T[u][p].edges(), get_cost(
                            T[u][p], G), len(Q)
                    Q = update_queue(Q, cg, u, p)
                    if len(p) == len(P) and cg < min_cost:#迭代最小的cost
                        min_cost = cg
                    # print "Min cost updated by growth ",min_cost
            else:  # If T[u][p] does not exist, create T[u][p]=merged graph
                # if gflag==1:
                if u not in T:
                    T[u] = {}
                T[u][p] = g
                flag = 1
                # if flag==1:
                heappush(Q, (cg, u, p))

                if verbose:
                    print "\nGrowth created new root, query", u, p, cg, T[u][p].nodes(), T[u][p].edges(), get_cost(
                        T[u][p], G), len(Q)
                if len(p) == len(P) and cg < min_cost:
                    min_cost = cg
                # print "Min cost created by growth ",min_cost

        # MERGE
        p1 = p #corner v对应的problem node

        all_p2 = set()
        for p2 in T[v]:#T[v]中的problem node 
            all_p2.add(p2)

        for p2 in all_p2:  # because T[v] changes during iteration
            if len(p1.intersection(p2)) == 0:
                # mflag=0
                g = merge_graph(T[v][p1], T[v][p2]) #v对应的problem的所有nodes融合起来，变成一个v对problem的几个值
                cg = get_cost(g, G)
                p = frozenset(p1.union(p2))
                if p in T[v]:  # all_p2:
                    cp = get_cost(T[v][p], G) #求最小cost
                    if cg < cp:
                        if verbose:
                            print "\nBefore merge updated", v, p, cg, cp, T[v][p].nodes(), T[v][p].edges(), get_cost(
                                T[v][p], G), len(Q)
                        T[v][p] = g
                        # heappush(Q,(cg,v,pp))
                        Q = update_queue(Q, cg, v, p)#更新Q，cost
                        if verbose:
                            print "\nmerge updated", v, p1, p2, p, cg, cp, T[v][p].nodes(), T[v][p].edges(), get_cost(
                                T[v][p], G), len(Q)
                        if len(p) == len(P) and cg < min_cost:
                            min_cost = cg
                        # print "Min cost updated by merge ",min_cost
                else:
                    # if mflag==1:
                    T[v][p] = g
                    heappush(Q, (cg, v, p))
                    if len(p) > merge_cov:
                        merge_cov = len(p)
                        if verbose:
                            print "merge length increased to ", len(p)
                    if len(p) == len(P):
                        if verbose:
                            print "\nmerge created new query", v, p, cg, T[v][p].nodes(), T[v][p].edges(), get_cost(
                                T[v][p], G), len(Q)
                    if len(p) == len(P) and cg < min_cost:
                        min_cost = cg
                    # print "Min cost created by merge ",min_cost
                    if verbose:
                        print "\nmerge created new query", v, p, cg, T[v][p].nodes(), T[v][p].edges(), get_cost(T[v][p],
                                                                                                                G), len(
                            Q)
                        print "v p1", v, p1, T[v][p1].nodes(), T[v][p1].edges(), get_cost(T[v][p1], G), len(Q)
                        print "v p2", v, p2, T[v][p2].nodes(), T[v][p2].edges(), get_cost(T[v][p2], G), len(Q)
    # print "\n\nGST Set before returning ",GST_set
    return GST_set

In [36]:
def check_history(pop_hist, v, p):
    if v in pop_hist:
        if p in pop_hist[v]:
            if verbose:
                print "\nsame item popped"
            return 0
    return 1

In [37]:
def grow_graph(g1, v, u):#加入g1的边和节点，加入v和u, 节点和边。g1: corner node v's graph. v:corner node. u: v's neibor
    g = nx.Graph()
    for n in g1.nodes():
        g.add_node(n) ##add g1's node & edge
    for (n1, n2) in g1.edges():
        g.add_edge(n1, n2)
    # flag=0
    if u not in g.nodes(): #add neibors' edges
        g.add_node(u)
        g.add_edge(v, u)  # to keep it a tree #access weight from G if needed
    # flag=1

    return g  # ,flag

In [38]:
def update_queue(Q, cg, u, p):
    for i in range(0, len(Q)):
        if Q[i][1] == u and Q[i][2] == p:
            # if verbose:
            # print "i th ",i,Q[i]
            Q[i] = (cg, u, p)
            heapify(Q)
            break
    return Q

In [39]:
def merge_graph(g1, g2):
    g = nx.Graph()
    for n in g1.nodes():
        g.add_node(n)
    for (n1, n2) in g1.edges():
        g.add_edge(n1, n2)
    # flag=0
    for n in g2.nodes():
        if n not in g.nodes():
            g.add_node(n)
        # flag=1
    for (n1, n2) in g2.edges():
        if (n1, n2) not in g.edges() and (n2, n1) not in g.edges():
            g.add_edge(n1, n2)
        # flag=1
    return g

In [40]:
def save_GST(GST_set, v, g1, corner, G): #expanding node by 'predicate', add v, if it is a predict, add 'entity' & 'triple' neiber(not g1's neibor, not in corner)
    g = nx.Graph()
    edge_added = 0
    if chain_join_flag == 1:
        for n in g1.nodes(): #v: node in corner. g1:corner v's graph
            g.add_node(n)
            nn = n.split(':')
            if nn[1] == 'Predicate' and n in corner:  # Add neighbour entities from SPO which are not corner stones
                for nb in G.neighbors(n): 
                    #当g1中的点n是predicate并且在corner中，获取G中n的邻居数据
                    #在big text中，获取对应的clause的子节点数据，并加入g中
                    data = G.get_edge_data(n, nb)
                    if nb.split(':')[1] == 'Entity' and nb not in g1.neighbors(n) and nb not in corner and data[
                        'etype'] == 'Triple':
                        g.add_node(nb)
                        g.add_edge(n, nb)
                        edge_added += 1

    for (n1, n2) in g1.edges():#加入g1中的edges
        g.add_edge(n1, n2)

    pot_ans_flag = 0
    for n in g.nodes():
        nn = n.split(':')
        if nn[1] == 'Entity' and n not in corner:  # At least one non-cornerstone entity node
            pot_ans_flag = 1

    flag = 1
    for (v2, g2) in GST_set:
        if set(g.edges()) == set(g2.edges()):
            # if g==g2:
            flag = 0
            break

    # print "Cost while saving ",get_cost(g,G),flag,pot_ans_flag,edge_added,len(GST_set)
    # for (n1,n2) in g.edges():
    # data=G.get_edge_data(n1,n2)
    # print "Saved Tree edge ",n1,n2,data['weight'],data['wlist'],data['etype']
    if flag == 1 and pot_ans_flag == 1:
        GST_set.append((v, g))#v: node in corner, g: expanding graph by v.(v and its neighbors)

    return GST_set, flag, pot_ans_flag

In [41]:
v

u'born in:Predicate:5'

In [42]:
T[v]#T, T -> v(node in corner & neibor):p(problem):graph(one node)

{frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d15ed90>}

In [43]:
T

{u'11 steve jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d1380d0>},
 u'1985 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b090>},
 u'1986 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138b10>},
 u'2003 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b250>},
 u'2009 jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d11b5d0>},
 u'actor steve carell:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138c10>},
 u'adoption jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138e50>},
 u'after jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d138190>},
 u'apple co-founder 06 oct 2011 steve jobs:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d15e090>},
 u'apple co-founder steve jobs:Entity': {frozenset({'steve jobs'}): <n

In [44]:
Q

[(0.0, u'11 steve jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'2009 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'1985 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'after jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'apple jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'1986 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'2003 jobs:Entity', frozenset({'steve jobs'})),
 (0.0, u'birth mother out:Predicate:1', frozenset({'born'})),
 (0.0, u'birth after:Predicate:1', frozenset({'born'})),
 (0.0, u'born in:Predicate:3', frozenset({'born'})),
 (0.0, u'born in:Predicate:5', frozenset({'born'})),
 (0.0, u'born on:Predicate:3', frozenset({'born'})),
 (0.0, u'actor steve carell:Entity', frozenset({'steve jobs'})),
 (0.0, u'adoption jobs:Entity', frozenset({'steve jobs'})),
 (0.0,
  u'apple co-founder 06 oct 2011 steve jobs:Entity',
  frozenset({'steve jobs'})),
 (0.0, u'born in:Predicate:4', frozenset({'born'})),
 (0.0, u'book steve jobs 2011:Entity', frozenset({'steve

In [45]:
no_GST

50

In [46]:
corner

{u'11 steve jobs:Entity': 'steve jobs',
 u'1985 jobs:Entity': 'steve jobs',
 u'1986 jobs:Entity': 'steve jobs',
 u'2003 jobs:Entity': 'steve jobs',
 u'2009 jobs:Entity': 'steve jobs',
 u'actor steve carell:Entity': 'steve jobs',
 u'adoption jobs:Entity': 'steve jobs',
 u'after jobs:Entity': 'steve jobs',
 u'apple co-founder 06 oct 2011 steve jobs:Entity': 'steve jobs',
 u'apple co-founder steve jobs:Entity': 'steve jobs',
 u'apple jobs:Entity': 'steve jobs',
 u'biological father steve jobs:Entity': 'steve jobs',
 u'birth after:Predicate:1': 'born',
 u'birth family during:Predicate:1': 'born',
 u'birth father:Predicate:1': 'born',
 u'birth mother out:Predicate:1': 'born',
 u'book steve jobs 2011:Entity': 'steve jobs',
 u'book steve jobs:Predicate:1': 'steve jobs',
 u'born in:Predicate:1': 'born',
 u'born in:Predicate:2': 'born',
 u'born in:Predicate:3': 'born',
 u'born in:Predicate:4': 'born',
 u'born in:Predicate:5': 'born',
 u'born in:Predicate:6': 'born',
 u'born in:Predicate:7': 'bo

In [47]:
P

{'born', 'steve jobs'}

In [48]:
GST_set = get_GST(Q, T, P, G, no_GST, corner, verbose)
#corner的v周围的点加进去，加gst number次， 每次要更新Q的cost，每次选最小的cost的v加入（第一次基本随机，因为cost是0）

In [49]:
Q

[(1.1102230246251565e-16, u'life with:Predicate:1', frozenset({'steve jobs'})),
 (1.1102230246251565e-16,
  u'life with:Predicate:1',
  frozenset({'born', 'steve jobs'})),
 (1.1102230246251565e-16, u'life with:Predicate:1', frozenset({'born'})),
 (1.1102230246251565e-16,
  u'refused:Predicate:2',
  frozenset({'born', 'steve jobs'})),
 (1.1102230246251565e-16, u'made in:Predicate:1', frozenset({'born'})),
 (1.1102230246251565e-16, u'refused:Predicate:2', frozenset({'born'})),
 (1.1102230246251565e-16,
  u'made in:Predicate:1',
  frozenset({'born', 'steve jobs'})),
 (1.1102230246251565e-16,
  u'take:Predicate:1',
  frozenset({'born', 'steve jobs'})),
 (0.053345704566829411, u'son of:Predicate:1', frozenset({'born'})),
 (1.1102230246251565e-16, u'made in:Predicate:1', frozenset({'steve jobs'})),
 (1.1102230246251565e-16,
  u'made:Predicate:1',
  frozenset({'born', 'steve jobs'})),
 (1.1102230246251565e-16,
  u'started:Predicate:3',
  frozenset({'born', 'steve jobs'})),
 (0.090875333099419

In [50]:
T

{u'graphisoft park:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d6dcfd0>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d856750>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x24d8567d0>},
 u'paul:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d594f50>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d2cecd0>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x24d613090>},
 u'consider:Predicate:1': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d66c850>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d9346d0>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x1eb146850>},
 u'private spirituality:Predicate:1': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x1e7abaf10>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24e1d43d0>,
  frozenset({'bor

In [51]:
GST_set#

[(u'1958:Entity', <networkx.classes.graph.Graph at 0x24d2fa750>),
 (u'apple co-founder jobs:Entity',
  <networkx.classes.graph.Graph at 0x24d2ced90>),
 (u'born in:Predicate:2', <networkx.classes.graph.Graph at 0x24d372550>),
 (u'1931:Entity', <networkx.classes.graph.Graph at 0x24d372950>),
 (u'born in:Predicate:3', <networkx.classes.graph.Graph at 0x24d372790>),
 (u'august 1995:Entity', <networkx.classes.graph.Graph at 0x24d372c50>),
 (u'born in:Predicate:4', <networkx.classes.graph.Graph at 0x24d372d10>),
 (u'born in:Predicate:5', <networkx.classes.graph.Graph at 0x24d372fd0>),
 (u'1998:Entity', <networkx.classes.graph.Graph at 0x24d397190>),
 (u'born in:Predicate:6', <networkx.classes.graph.Graph at 0x24d397050>),
 (u'born in:Predicate:7', <networkx.classes.graph.Graph at 0x24d397310>),
 (u'1955:Entity', <networkx.classes.graph.Graph at 0x24d397510>),
 (u'born in:Predicate:8', <networkx.classes.graph.Graph at 0x24d397450>),
 (u'born into:Predicate:1', <networkx.classes.graph.Graph at

In [52]:
GST_set[0]# node, node-problem 相关的network

(u'1958:Entity', <networkx.classes.graph.Graph at 0x24d2fa750>)

In [53]:
GST_set[0][1]

In [54]:
GST_set[0][1].nodes()

[u'joanne simpson:Entity',
 u'parents:Entity',
 u'born in:Predicate:1',
 u'abdulfattah john jandali:Entity',
 u'adoption jobs:Entity',
 u'1958:Entity',
 u'born:Predicate:3',
 u'two university of wisconsin graduate students:Entity']

In [55]:
GST_set[0][1].edges()

[(u'joanne simpson:Entity', u'born:Predicate:3'),
 (u'parents:Entity', u'born:Predicate:3'),
 (u'born in:Predicate:1', u'born:Predicate:3'),
 (u'born in:Predicate:1', u'1958:Entity'),
 (u'abdulfattah john jandali:Entity', u'born:Predicate:3'),
 (u'adoption jobs:Entity', u'born:Predicate:3'),
 (u'born:Predicate:3',
  u'two university of wisconsin graduate students:Entity')]

In [56]:
GST_set_list = GST_set
GST_set = set(GST_set)

In [57]:
GST_set_list

[(u'1958:Entity', <networkx.classes.graph.Graph at 0x24d2fa750>),
 (u'apple co-founder jobs:Entity',
  <networkx.classes.graph.Graph at 0x24d2ced90>),
 (u'born in:Predicate:2', <networkx.classes.graph.Graph at 0x24d372550>),
 (u'1931:Entity', <networkx.classes.graph.Graph at 0x24d372950>),
 (u'born in:Predicate:3', <networkx.classes.graph.Graph at 0x24d372790>),
 (u'august 1995:Entity', <networkx.classes.graph.Graph at 0x24d372c50>),
 (u'born in:Predicate:4', <networkx.classes.graph.Graph at 0x24d372d10>),
 (u'born in:Predicate:5', <networkx.classes.graph.Graph at 0x24d372fd0>),
 (u'1998:Entity', <networkx.classes.graph.Graph at 0x24d397190>),
 (u'born in:Predicate:6', <networkx.classes.graph.Graph at 0x24d397050>),
 (u'born in:Predicate:7', <networkx.classes.graph.Graph at 0x24d397310>),
 (u'1955:Entity', <networkx.classes.graph.Graph at 0x24d397510>),
 (u'born in:Predicate:8', <networkx.classes.graph.Graph at 0x24d397450>),
 (u'born into:Predicate:1', <networkx.classes.graph.Graph at

In [58]:
GST_set_list[0]

(u'1958:Entity', <networkx.classes.graph.Graph at 0x24d2fa750>)

In [59]:
answer_type = open(f4, 'r')
ans_type = set()
for line in answer_type:
    ans_type.add(line.strip())

In [60]:
answer_type

<open file '/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/question_type.txt', mode 'r' at 0x24bc9e4b0>

In [61]:
ans_type

{'date'}

In [62]:
match_flag = 0
unique_nodes = set()
tree_pot = {}  # trees with potential answers
tc = 0
ans_tree = []

In [63]:
if Type_Filtering_flag == 1:
    candidate_set = set()
    # Check number of candidate answers
    for (v, T) in GST_set:
        for n in T.nodes():
            nn = n.split(':')
            if nn[1] == 'Entity' and n not in corner:  # Non entities and Cornernerstones are removed
                candidate_set.add(n)

    if len(candidate_set) >= 10:
        type_threshold = threshold
    else:
        type_threshold = -1
else:
    if Type_Filtering_flag == 0:
        type_threshold = -1
    else:
        type_threshold = threshold


In [64]:
v#corner的点

u'born in:Predicate:5'

In [65]:
T#(v, T) in GST_set

{u'graphisoft park:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d6dcfd0>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d856750>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x24d8567d0>},
 u'paul:Entity': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d594f50>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d2cecd0>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x24d613090>},
 u'consider:Predicate:1': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x24d66c850>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24d9346d0>,
  frozenset({'born',
             'steve jobs'}): <networkx.classes.graph.Graph at 0x1eb146850>},
 u'private spirituality:Predicate:1': {frozenset({'steve jobs'}): <networkx.classes.graph.Graph at 0x1e7abaf10>,
  frozenset({'born'}): <networkx.classes.graph.Graph at 0x24e1d43d0>,
  frozenset({'bor

In [68]:
gt1 = set(['unknown'])

In [69]:
def get_type_simi(n0, G, ans_type, gdict):#compare n0's all type in G, with answer type, pick the max_similar one
    veclen = 300 #比较n0的type和ans的type
    term_types = set()
    for (n1, n2) in G.edges():
        if n0 == n2 and n1.split(':')[1] == 'Type':#在G中， n0=n2, n1 is a type node，get type name
            term_types.add(n1.split(':')[0])
        else:
            if n0 == n1 and n2.split(':')[1] == 'Type':#在G中， n0=n1, n2 is a type node
                term_types.add(n2.split(':')[0])

    # if verbose:
    # print "Answer type, term type ",ans_type,term_types
    # gdict=word_vectors

    if len(term_types) == 0 or len(ans_type) == 0:  # if answer does not have type or node n0 does not have type, keep n0
        return 1.0
#Embedding similarity can be avg of vectors (0)
#or maximum match (1), Cosine_threshold gives the corresponding threshold:
#Embedding_Similarity: 1
#Cosine_threshold: 0.75
    if MAX_MATCH == 1: 
        maxval = 0.0

        for n1 in term_types:
            for n2 in ans_type:
                val = cosine_similarity_MAX_MATCH(n1, n2, gdict)
                if val > maxval:
                    maxval = val

    else:
        t_dict = {}
        a_dict = {}

        for n in term_types:
            nw1 = n.split()
            avec = np.zeros(veclen)
            c = 0.0
            for el in nw1:
                if el in gdict and el.lower() not in stop_list:
                    avec = np.add(avec, np.array(gdict[el]))
                    c += 1.0
            if c > 0:
                avec = np.divide(avec, c)

            t_dict[n] = avec.tolist()

        for n in ans_type:
            nw1 = n.split()
            avec = np.zeros(veclen)
            c = 0.0
            for el in nw1:
                if el in gdict and el.lower() not in stop_list:
                    avec = np.add(avec, np.array(gdict[el]))
                    c += 1.0
            if c > 0:
                avec = np.divide(avec, c)

            a_dict[n] = avec.tolist()

        maxval = 0.0

        for n1 in t_dict:
            for n2 in a_dict:
                val = cosine_similarity(t_dict[n1], a_dict[n2])
                if val > maxval:
                    maxval = val
    return maxval

In [70]:
def cosine_similarity_MAX_MATCH(a, b, gdict):
    a = a.lower()
    aw1 = a.replace('-', ' ').split()
    b = b.lower()
    bw1 = b.replace('-', ' ').split()

    max_match = -1
    for el1 in aw1:
        if el1 in gdict and el1 not in stop_list:
            avec = gdict[el1]
            for el2 in bw1:
                if el2 in gdict and el2 not in stop_list:
                    bvec = gdict[el2]
                    val = cosine_similarity(avec, bvec)
                    if val > max_match:
                        max_match = val
    return max_match


In [71]:
stop_list = {'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'aren\'t',
             'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can\'t',
             'cannot', 'could', 'couldn\'t', 'did', 'didn\'t', 'do', 'does', 'doesn\'t', 'doing', 'don\'t', 'down',
             'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn\'t', 'has', 'hasn\'t', 'have', 'haven\'t',
             'having', 'he', 'he\'d', 'he\'ll', 'he\'s', 'her', 'here', 'here\'s', 'hers', 'herself', 'him', 'himself',
             'his', 'how', 'how\'s', 'i', 'i\'d', 'i\'ll', 'i\'m', 'i\'ve', 'if', 'in', 'into', 'is', 'isn\'t', 'it',
             'it\'s', 'its', 'itself', 'let\'s', 'me', 'more', 'most', 'mustn\'t', 'my', 'myself', 'no', 'nor', 'not',
             'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours	ourselves', 'out', 'over',
             'own', 'same', 'shan\'t', 'she', 'she\'d', 'she\'ll', 'she\'s', 'should', 'shouldn\'t', 'so', 'some',
             'such', 'than', 'that', 'that\'s', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there',
             'there\'s', 'these', 'they', 'they\'d', 'they\'ll', 'they\'re', 'they\'ve', 'this', 'those', 'through',
             'to', 'too', 'under', 'until', 'up', 'very', 'was', 'wasn\'t', 'we', 'we\'d', 'we\'ll', 'we\'re', 'we\'ve',
             'were', 'weren\'t', 'what', 'what\'s', 'when', 'when\'s', 'where', 'where\'s', 'which', 'while', 'who',
             'who\'s', 'whom', 'why', 'why\'s', 'with', 'won\'t', 'would', 'wouldn\'t', 'you', 'you\'d', 'you\'ll',
             'you\'re', 'you\'ve', 'your', 'yours', 'yourself', 'yourselves'}

In [72]:
def cosine_similarity(a, b):
    s1 = 0.0
    s2 = 0.0
    s3 = 0.0
    if len(a) != len(b):
        return 0.0
    for i in range(0, len(a)):
        s1 += a[i] * b[i]
        s2 += a[i] * a[i]
        s3 += b[i] * b[i]
    if s2 > 0 and s3 > 0:
        val = (s1 / (math.sqrt(s2) * math.sqrt(s3)))
        val_norm = (val + 1.0) / 2.0
        return val_norm
    else:
        return 0

In [73]:
for (v, T) in GST_set:#查看type值，相似度大的， corner中没有的点放入tree_pot
    # if verbose:
    # print "\n\nObtained GST-->",tc,v.encode('utf-8'),T.nodes(),T.edges(),get_cost(T,G)
    # for (n,m) in T.edges():
    # data1=G.get_edge_data(n,m)
    # print "Edges ",n,m,data1['weight'],data1['etype'],data1['wlist']
    # print "\nPotential answers filtered by type -->"

    tree_pot[(v, T)] = set()#v:corner T:corner中的v及其周围的邻居组成的graph

    for n in T.nodes():#查看邻居中的点
        nn = n.split(':')

        for gt in gt1:#check if get answer
            if len(nn) > 0:
                if nn[0] == (gt.lower()).decode('utf-8'):
                    match_flag = 1
            else:
                if n == (gt.lower()).decode('utf-8'):
                    match_flag = 1

        if nn[1] == 'Entity' and n not in corner:  # Non entities and Cornernerstones are removed
            type_chk = get_type_simi(n, G, ans_type, gdict)#得到最大的type相似度值
            if verbose:
                print "\nType match ", n, type_chk

            if type_chk >= type_threshold:  # (len(ans_type)>0 and type_chk>=0.5) or (len(ans_type)==0 and type_chk>=-1):       #Answer type non matching are removed
                tree_pot[(v, T)].add(n)#如果type的相似度大于一定值，则把这些T中的邻居节点加入到tree中，tree_pot[(v, T)]有多个值
            # if verbose:
            # print n.encode('utf-8')

    '''
    for n in T.nodes():
        unique_nodes.add(n.encode('utf-8'))
        nn=n.split(':')
        if nn[0]==ans:
            ans_tree.append(tc)
    '''

    tc += 1

In [81]:
type_chk

1.0

In [76]:
v

u'talking about:Predicate:1'

In [77]:
T.nodes()

[u'joanne simpson:Entity',
 u'born in:Predicate:7',
 u'parents:Entity',
 u'adoption jobs:Entity',
 u'abdulfattah john jandali:Entity',
 u'born in:Predicate:1',
 u'san francisco:Entity',
 u'talking about:Predicate:1',
 u'steve jobs:Entity',
 u'_ page 165:Entity',
 u'born:Predicate:3',
 u'_ page _ title february 24 , 1955:Entity',
 u'1958:Entity',
 u'two university of wisconsin graduate students:Entity']

In [78]:
T.edges()

[(u'joanne simpson:Entity', u'born:Predicate:3'),
 (u'born in:Predicate:7', u'_ page 165:Entity'),
 (u'born in:Predicate:7', u'steve jobs:Entity'),
 (u'born in:Predicate:7', u'_ page _ title february 24 , 1955:Entity'),
 (u'born in:Predicate:7', u'born in:Predicate:1'),
 (u'born in:Predicate:7', u'san francisco:Entity'),
 (u'parents:Entity', u'born:Predicate:3'),
 (u'adoption jobs:Entity', u'born:Predicate:3'),
 (u'abdulfattah john jandali:Entity', u'born:Predicate:3'),
 (u'born in:Predicate:1', u'born:Predicate:3'),
 (u'born in:Predicate:1', u'1958:Entity'),
 (u'talking about:Predicate:1', u'steve jobs:Entity'),
 (u'born:Predicate:3',
  u'two university of wisconsin graduate students:Entity')]

In [79]:
if verbose:
    print "\n\ntrees before merging --->", tree_pot

match = {}

In [80]:
def issubseq(n1, n2):#检查n1是不是n2的原型。n2是不是经过了一定的时态语态变化
    nw1 = (n1.split(':'))[0].split()
    nw2 = (n2.split(':'))[0].split()

    i = 0
    flag = 0
    for j in range(0, len(nw2)):
        if nw1[i].lower() == nw2[j].lower():
            i += 1
            if i == len(nw1):
                flag = 1
                break
    if flag == 1:
        return 1
    else:
        return 0

In [82]:
# Answer merging #（在big-text中，entity是已经merg过的唯一值）
for (v1, T1) in tree_pot:
    for n1 in tree_pot[(v1, T1)]:
        for (v2, T2) in tree_pot:
            for n2 in tree_pot[(v2, T2)]:
                flag = 0
                if n1 != n2:
                    if (n1, n2) in G.edges():
                        data = G.get_edge_data(n1, n2) 
                        #检查corner的点v，扩展出来的T中间是不是有相同的点（tree_pot中是否有相同），
                        #这里的entity_alignment就是相同的点
                        if data['etype'] == 'Entity_Alignment':
                            flag = 1

                    else:
                        if (n2, n1) in G.edges():
                            data = G.get_edge_data(n2, n1)
                            if data['etype'] == 'Entity_Alignment':
                                flag = 1

                    if flag == 1:  # and len(n2)>len(n1): #merging into the longer answer
                        if n1 not in match: #如果相等就放进match中，match[n1]=n2，match[n2]=n1
                            match[n1] = set()
                        match[n1].add(n2)
                        if n2 not in match:
                            match[n2] = set()
                        match[n2].add(n1)

                    else:  # Check Subsequence
                        if issubseq(n1, n2) or issubseq(n2, n1):#n1，n2看为类似，如果经过了时态语态变化
                            if n1 not in match:
                                match[n1] = set()
                            match[n1].add(n2)
                            if n2 not in match:
                                match[n2] = set()
                            match[n2].add(n1)
    # print "\nMatch after iteration with n1 ",n1
    # for x in match:
    # print x, match[x]

KeyboardInterrupt: 

In [ ]:
match

In [ ]:
tree_pot#type相似度在一定值以上的GST_set 
#tree_pot[(v, T)]
#id:(v, T), v：corner中的点，T：v周围的点组成的graph
#value:T中的点

In [ ]:
# Find unique sets of answers
ans_set = set()#把tree_pot中（选出的type大于一定值的点），每个node对应的值选出来，排除相同的点
done = set()
for (v1, T1) in tree_pot:#(v1, T1)是id
    for n1 in tree_pot[(v1, T1)]:#n是node
        if n1 not in done:
            curr = set()
            q = Queue.Queue()
            q.put(n1)#node 放入q中
            while not q.empty():
                nn = q.get()#从队列里取数据
                curr.add(nn)
                done.add(nn)
                if nn in match:
                    for n2 in match[nn]:
                        if n2 not in curr and n2 not in q.queue:
                            q.put(n2)#向队列里放数据

            ans_set.add(frozenset(curr))

In [ ]:
q

In [ ]:
ans_set

In [ ]:
def get_cornerstone_weight(T, G, corner):
    w = 0.0
    for n in T.nodes():
        if n in corner:
            w += G.node[n]['weight']
    return w

In [ ]:
def get_edge_sentid_GST(T, G):
    sent = set()
    for (n1, n2) in T.edges():
        if (n1, n2) in G.edges():
            data = G.get_edge_data(n1, n2)
        else:
            data = G.get_edge_data(n2, n1)

        for i in range(0, len(data['dtitle'])):
            sent.add((data['dtitle'][i], data['sid'][i]))
    return sent

In [ ]:
def get_corner_wt(ans_set, tree_pot, G, corner, gt1):
    # Add weights to answers
    done = set()
    answer_list = []
    candidate_match_flag = 0
    for ans in ans_set:#在ans_set中
        w = 0.0
        support = set()
        for (v1, T1) in tree_pot:
            for n1 in tree_pot[(v1, T1)]:
                if n1 in ans:
                    w += get_cornerstone_weight(T1, G, corner)
                    sent = get_edge_sentid_GST(T1, G)
                    support = support.union(sent)#支撑答案的sentence

        anslist = list(ans)
        s = anslist[0]
        for i in range(1, len(anslist)):
            s = s + '|' + anslist[i] #把所有anslist中的值连起来作为1个值
        answer_list.append((s, support, w))

        for gt in gt1:
            for i in range(0, len(anslist)):
                nnc1 = anslist[i].split(':')
                # if nnc1[0]==gt.lower():
                #	candidate_match_flag=1
                if len(nnc1) > 0:
                    if nnc1[0] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1
                else:
                    if anslist[i] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1

    answer_list = sorted(answer_list, key=lambda x: x[2], reverse=True)
    top_match_flag = 0
    for i in range(0, 5):
        if i >= len(answer_list):
            break
        nnc1 = answer_list[0][0].split(':')

        ans1 = answer_list[i][0].split('|')
        for s in ans1:
            s = s.split(':')[0]
            s = s.strip()
            s = s.strip(',')
            # s=s.encode('utf-8')

            for gt in gt1:
                if s == (gt.lower()).decode('utf-8'):
                    top_match_flag = 1
    if top_match_flag == 0:
        for i in range(4, len(answer_list)):
            if answer_list[i][2] == answer_list[4][2]:  # same score as the 5th one
                ans1 = answer_list[i][0].split('|')
                for s in ans1:
                    s = s.split(':')[0]
                    s = s.strip()
                    s = s.strip(',')
                    # s=s.encode('utf-8')

                    for gt in gt1:
                        if s == (gt.lower()).decode('utf-8'):
                            top_match_flag = 1
            else:
                break
    # if verbose:
    if verbose:
        print "\n\nFinal answer ranking --->", len(answer_list)  # ,answer_list
    return answer_list, candidate_match_flag, top_match_flag

In [ ]:
answer_list, candidate_match_flag1, top_match_flag1 = get_corner_wt(ans_set, tree_pot, G, corner, gt1)
str1 = f5 + '_node_wt'
# pickle.dump(answer_list, open(str1, 'w'))

In [ ]:
def get_tree_cost(ans_set, tree_pot, G, corner, gt1):
    # Add weights to answers
    done = set()
    answer_list = []
    candidate_match_flag = 0
    for ans in ans_set:
        w = 0.0
        support = set()
        for (v1, T1) in tree_pot:
            for n1 in tree_pot[(v1, T1)]:
                if n1 in ans:
                    cost_tree = float(get_cost(T1, G))
                    if cost_tree > 0.0:
                        w += 1.0 / cost_tree  # get_cornerstone_weight(T1, G, corner)
                    sent = get_edge_sentid_GST(T1, G)
                    support = support.union(sent)

        anslist = list(ans)
        s = anslist[0]
        for i in range(1, len(anslist)):
            s = s + '|' + anslist[i]
        answer_list.append((s, support, w))

        for gt in gt1:
            for i in range(0, len(anslist)):
                nnc1 = anslist[i].split(':')
                # if nnc1[0]==gt.lower():
                #	candidate_match_flag=1
                if len(nnc1) > 0:
                    if nnc1[0] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1
                else:
                    if anslist[i] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1

    answer_list = sorted(answer_list, key=lambda x: x[2], reverse=True)
    top_match_flag = 0
    for i in range(0, 5):
        if i >= len(answer_list):
            break
        nnc1 = answer_list[0][0].split(':')
        ans1 = answer_list[i][0].split('|')
        for s in ans1:
            s = s.split(':')[0]
            s = s.strip()
            s = s.strip(',')
            # s=s.encode('utf-8')

            for gt in gt1:
                if s == (gt.lower()).decode('utf-8'):
                    top_match_flag = 1
    if top_match_flag == 0:
        for i in range(4, len(answer_list)):
            if answer_list[i][2] == answer_list[4][2]:  # same score as the 5th one
                ans1 = answer_list[i][0].split('|')
                for s in ans1:
                    s = s.split(':')[0]
                    s = s.strip()
                    s = s.strip(',')
                    # s=s.encode('utf-8')

                    for gt in gt1:
                        if s == (gt.lower()).decode('utf-8'):
                            top_match_flag = 1
            else:
                break
    if verbose:
        print "\n\nFinal answer ranking --->", len(answer_list)  # ,answer_list
    return answer_list, candidate_match_flag, top_match_flag


In [ ]:
answer_list, candidate_match_flag2, top_match_flag2 = get_tree_cost(ans_set, tree_pot, G, corner, gt1)
str1 = f5 + '_tree_cost'
# pickle.dump(answer_list, open(str1, 'w'))

In [ ]:
def get_tree_count(ans_set, tree_pot, G, corner, gt1):
    # Add weights to answers
    done = set()
    answer_list = []
    candidate_match_flag = 0
    for ans in ans_set:
        w = 0.0
        support = set()
        for (v1, T1) in tree_pot:
            for n1 in tree_pot[(v1, T1)]:
                if n1 in ans:
                    w += 1.0  # get_cornerstone_weight(T1, G, corner)
                    sent = get_edge_sentid_GST(T1, G)
                    support = support.union(sent)

        anslist = list(ans)
        s = anslist[0]
        for i in range(1, len(anslist)):
            s = s + '|' + anslist[i]
        answer_list.append((s, support, w))

        for gt in gt1:
            for i in range(0, len(anslist)):
                nnc1 = anslist[i].split(':')
                # if nnc1[0]==gt.lower():
                #	candidate_match_flag=1
                if len(nnc1) > 0:
                    if nnc1[0] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1
                else:
                    if anslist[i] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1

    answer_list = sorted(answer_list, key=lambda x: x[2], reverse=True)
    top_match_flag = 0
    for i in range(0, 5):
        if i >= len(answer_list):
            break
        nnc1 = answer_list[0][0].split(':')

        ans1 = answer_list[i][0].split('|')
        for s in ans1:
            s = s.split(':')[0]
            s = s.strip()
            s = s.strip(',')
            # s=s.encode('utf-8')

            for gt in gt1:
                if s == (gt.lower()).decode('utf-8'):
                    top_match_flag = 1
    if top_match_flag == 0:
        for i in range(4, len(answer_list)):
            if answer_list[i][2] == answer_list[4][2]:  # same score as the 5th one
                ans1 = answer_list[i][0].split('|')
                for s in ans1:
                    s = s.split(':')[0]
                    s = s.strip()
                    s = s.strip(',')
                    # s=s.encode('utf-8')

                    for gt in gt1:
                        if s == (gt.lower()).decode('utf-8'):
                            top_match_flag = 1
            else:
                break
    if verbose:
        print "\n\nFinal answer ranking --->", len(answer_list)  # ,answer_list
    return answer_list, candidate_match_flag, top_match_flag


In [ ]:
answer_list, candidate_match_flag3, top_match_flag3 = get_tree_count(ans_set, tree_pot, G, corner, gt1)
str1 = f5 + '_tree_count'
# pickle.dump(answer_list, open(str1, 'w'))

In [ ]:
def get_corner_dist(ans_set, tree_pot, G, corner, gt1):
    # Add weights to answers
    done = set()
    answer_list = []
    candidate_match_flag = 0
    for ans in ans_set:
        w = 0.0
        support = set()
        for (v1, T1) in tree_pot:
            for n1 in tree_pot[(v1, T1)]:
                if n1 in ans:
                    dist_corner = float(get_cornerstone_distance(T1, G, corner, n1))
                    if dist_corner > 0.0:
                        w += 1.0 / dist_corner  # get_cornerstone_weight(T1, G, corner)
                    sent = get_edge_sentid_GST(T1, G)
                    support = support.union(sent)

        anslist = list(ans)
        s = anslist[0]
        for i in range(1, len(anslist)):
            s = s + '|' + anslist[i]
        answer_list.append((s, support, w))

        for gt in gt1:
            for i in range(0, len(anslist)):
                nnc1 = anslist[i].split(':')
                # if nnc1[0]==gt.lower():
                #	candidate_match_flag=1
                if len(nnc1) > 0:
                    if nnc1[0] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1
                else:
                    if anslist[i] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1

    answer_list = sorted(answer_list, key=lambda x: x[2], reverse=True)
    top_match_flag = 0
    for i in range(0, 5):
        if i >= len(answer_list):
            break
        nnc1 = answer_list[0][0].split(':')

        ans1 = answer_list[i][0].split('|')
        for s in ans1:
            s = s.split(':')[0]
            s = s.strip()
            s = s.strip(',')
            # s=s.encode('utf-8')

            for gt in gt1:
                if s == (gt.lower()).decode('utf-8'):
                    top_match_flag = 1
    if top_match_flag == 0:
        for i in range(4, len(answer_list)):
            if answer_list[i][2] == answer_list[4][2]:  # same score as the 5th one
                ans1 = answer_list[i][0].split('|')
                for s in ans1:
                    s = s.split(':')[0]
                    s = s.strip()
                    s = s.strip(',')
                    # s=s.encode('utf-8')

                    for gt in gt1:
                        if s == (gt.lower()).decode('utf-8'):
                            top_match_flag = 1
            else:
                break
    if verbose:
        print "\n\nFinal answer ranking --->", len(answer_list)  # ,answer_list
    return answer_list, candidate_match_flag, top_match_flag


In [ ]:
answer_list, candidate_match_flag4, top_match_flag4 = get_corner_dist(ans_set, tree_pot, G, corner, gt1)
str1 = f5 + '_corner_dist'
# pickle.dump(answer_list, open(str1, 'w'))

In [ ]:
def get_corner_dist_wt(ans_set, tree_pot, G, corner, gt1):
    # Add weights to answers
    done = set()
    answer_list = []
    candidate_match_flag = 0
    for ans in ans_set:
        w = 0.0
        support = set()
        for (v1, T1) in tree_pot:
            for n1 in tree_pot[(v1, T1)]:
                if n1 in ans:
                    dist_corner = float(get_cornerstone_distance_wt(T1, G, corner, n1))
                    if dist_corner > 0.0:
                        w += 1.0 / dist_corner  # get_cornerstone_weight(T1, G, corner)
                    sent = get_edge_sentid_GST(T1, G)
                    support = support.union(sent)

        anslist = list(ans)
        s = anslist[0]
        for i in range(1, len(anslist)):
            s = s + '|' + anslist[i]
        answer_list.append((s, support, w))

        for gt in gt1:
            for i in range(0, len(anslist)):
                nnc1 = anslist[i].split(':')
                # if nnc1[0]==gt.lower():
                #	candidate_match_flag=1
                if len(nnc1) > 0:
                    if nnc1[0] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1
                else:
                    if anslist[i] == (gt.lower()).decode('utf-8'):
                        candidate_match_flag = 1

    answer_list = sorted(answer_list, key=lambda x: x[2], reverse=True)
    top_match_flag = 0
    for i in range(0, 5):
        if i >= len(answer_list):
            break
        nnc1 = answer_list[0][0].split(':')

        ans1 = answer_list[i][0].split('|')
        for s in ans1:
            s = s.split(':')[0]
            s = s.strip()
            s = s.strip(',')
            # s=s.encode('utf-8')

            for gt in gt1:
                if s == (gt.lower()).decode('utf-8'):
                    top_match_flag = 1
    if top_match_flag == 0:
        for i in range(4, len(answer_list)):
            if answer_list[i][2] == answer_list[4][2]:  # same score as the 5th one
                ans1 = answer_list[i][0].split('|')
                for s in ans1:
                    s = s.split(':')[0]
                    s = s.strip()
                    s = s.strip(',')
                    # s=s.encode('utf-8')

                    for gt in gt1:
                        if s == (gt.lower()).decode('utf-8'):
                            top_match_flag = 1
            else:
                break
    if verbose:
        print "\n\nFinal answer ranking --->", len(answer_list)  # ,answer_list
    return answer_list, candidate_match_flag, top_match_flag


In [ ]:
answer_list, candidate_match_flag5, top_match_flag5 = get_corner_dist_wt(ans_set, tree_pot, G, corner, gt1)
str1 = f5 + '_corner_dist_wt'
# pickle.dump(answer_list, open(str1, 'w'))

In [ ]:
len(G.nodes()), len(G.edges()), match_flag, candidate_match_flag1, top_match_flag1, candidate_match_flag2, top_match_flag2, candidate_match_flag3, top_match_flag3, candidate_match_flag4, top_match_flag4, candidate_match_flag5, top_match_flag5